In [1]:
from selenium import webdriver
from time import sleep
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import requests
from requests import get


In [39]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.agoda.com/search?guid=b1a692d2-2778-4f21-8b0f-a3ec2da44064&asq=wAVSsEhpvY7ouAHHoV7S5pufa9Vwpz6XltTHq4n%2B9gP8cmMVmoKIoDS8aO7amKeIQWPLS2pzy77DAK3HZR457%2BLl7bQeS52Yw0291h3LrUHYuDNjrl8YZToVXUOg2fGAyXlTJW6hRaWQKPrp2OQ0YmQ%2FDuO4Ycd3tHzVRGE03t84l8rVFgjkeyKuZdY6YmCX0Jgtxo5oTTGzUe9Cah0%2BGphM60DaWL%2BTQt%2FOYe5eJhk%3D&city=17188&tick=638516468699&locale=en-us&ckuid=1438d9c3-bee0-4670-97b5-2370492ba071&prid=0&gclid=CjwKCAjwl4yyBhAgEiwADSEjeLfV6860bwzC06adEeIKV12IgthSbnL8sKQsYUt8COxNxuybthEeMBoCZrEQAvD_BwE&currency=VND&correlationId=490d3585-0054-46d4-9647-894674ff176c&analyticsSessionId=8296938497717999712&pageTypeId=103&realLanguageId=1&languageId=1&origin=VN&stateCode=SG&cid=1891474&tag=11850829-04c0-4370-a730-98350cbceef9&userId=1438d9c3-bee0-4670-97b5-2370492ba071&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=78&currencyCode=VND&htmlLanguage=en-us&cultureInfoName=en-us&machineName=hk-pc-2g-acm-web-user-f6c778d5d-vvt6w&trafficGroupId=5&trafficSubGroupId=122&aid=82361&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Edge+%28Chromium%29+for+Windows&cdnDomain=agoda.net&checkIn=2024-05-19&checkOut=2024-05-20&rooms=1&adults=1&children=0&priceCur=VND&los=1&textToSearch=Phu+Quoc+Island&productType=-1&travellerType=0&familyMode=off&ds=Y5zSyi1rWT%2Flw2ua')
time.sleep(10)

In [40]:
all_hotel_data = []

In [41]:
i = 0
while i < 4:
    time.sleep(10)
    pre_height = driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(30)
        new_height = driver.execute_script('return document.body.scrollHeight')

        print('new_height: {}\npre_height: {}'.format(new_height, pre_height))

        if new_height == pre_height:
            print("While loop Done!!!")
            break
        pre_height = new_height
    soup = BeautifulSoup(driver.page_source, "html.parser")
    hotels = soup.find_all("h3", class_ = "sc-jrAGrp sc-kEjbxe eDlaBj dscgss")
    prices = soup.find_all("div", attrs={"data-element-name": "final-price"})
    ratings = soup.find_all("div", class_ = "Box-sc-kv6pi1-0 ggePrW")
    all_benefits_containers = soup.find_all("div", attrs={"data-element-name": "pill-container"})

    # Iterate over each hotel element and extract the required information
    for hotel, price, rating, benefits_container in zip(hotels, prices, ratings, all_benefits_containers):
        hotel_name = hotel.text.strip()
        price_text = price.get_text(strip=True)
        rating_text = rating.text.strip()
        benefit_list = benefit_list = [benefit.text.strip() for benefit in benefits_container.find_all("div", attrs={"data-element-name": "pill-each-item"})]

        # Only append if a price is available
        if price_text and rating_text:
            all_hotel_data.append({
                "Name": hotel_name,
                "Price": price_text,
                "Rating": rating_text,
                "Benefits": benefit_list,
                "Location": "Phú Quốc"
            })
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-selenium='pagination-next-btn']"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        next_page_button.click()
        time.sleep(20)  # Wait for the next page to load
    except TimeoutException:
        print("No more pages to load.")
        break  # Wait for the next page to load
    i += 1
     

new_height: 14034
pre_height: None
new_height: 26218
pre_height: 14034
new_height: 26251
pre_height: 26218
new_height: 26251
pre_height: 26251
While loop Done!!!
new_height: 6834
pre_height: None
new_height: 6834
pre_height: 6834
While loop Done!!!


KeyboardInterrupt: 

In [42]:
len(all_hotel_data)

28

In [43]:
import pandas as pd

# Convert the nested data to a DataFrame
hotel = pd.DataFrame(all_hotel_data, columns = ["Name",  "Price", "Rating", "Benefits", "Location"])

In [44]:
hotel

,Name,Price,Rating,Benefits,Location
0,Sunset Beach Resort & Spa,"₫1,020,243",8.8,"[Breakfast, Phòng tập miễn phí, +3]",Phú Quốc
1,Seashells Phu Quoc Hotel & Spa,"₫1,882,833",9,"[Breakfast, Phòng tập miễn phí, +6]",Phú Quốc
2,Mercury Phu Quoc Resort and Villas,"₫1,515,631",8.4,"[Breakfast, Phòng tập miễn phí, +7]",Phú Quốc
3,Valentina Resort & Spa Phu Quoc,"₫648,246",9,"[Breakfast, Welcome drink, Bãi đậu xe, +3]",Phú Quốc
4,Andochine Villas Resort & Spa – All Villas wit...,"₫3,733,905",9,"[Breakfast, Welcome drink, Bãi đậu xe, +4]",Phú Quốc
5,Venice Hotel Phu Quoc - Free Hon Thom Island W...,"₫1,281,769",9.9,"[Breakfast, Phòng tập miễn phí, +9]",Phú Quốc
6,Muong Thanh Luxury Phu Quoc Hotel,"₫1,106,654",8,"[Breakfast, Welcome drink, Bãi đậu xe, +3]",Phú Quốc
7,SOL By Melia Phu Quoc,"₫1,843,975",8.5,"[Breakfast, Bao gồm bữa tối, All Inclusive, +10]",Phú Quốc
8,Dusit Princess Moonrise Beach Resort,"₫1,962,774",9.2,"[Breakfast, Welcome drink, Bãi đậu xe, +3]",Phú Quốc
9,Coco Palm Beach Resort & Spa,"₫ 1,873,674",8.5,"[Breakfast, Welcome drink, +4]",Phú Quốc


In [46]:
# Specify the CSV file path
csv_file_path = 'hotel_data3.csv'

# Append data to CSV file
hotel.to_csv(csv_file_path, mode='a', index=False, header=not pd.io.common.file_exists(csv_file_path))


In [29]:
# Assume 'hotel' is your DataFrame variable name
hotel.to_csv('hotel_data3.csv', index=True)
